# Build The Neural Network

torch.nn namespace는 신경망을 구성하는 데 필요한 모든 구성 요소를 제공한다. PyTorch의 모든 모듈은 nn.Module의 subclass이다.

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

## Get Device for Training

In [2]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


## Define the Class

Neural network class는 nn.Module의 subclass로 만든다. 그리고 layers를 \_\_init\_\_에서 초기화한다. nn.Module의 subclass는 input data에 대한 처리를 forward 메서드에서 처리한다.

In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 18),
        )
        
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

NeuralNetwork 객체를 만들고 device로 옮긴다.

In [4]:
model = NeuralNetwork().to(device)
model

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=18, bias=True)
  )
)

model을 사용할 때에는 forward를 직접 호출하는 대신 model에 데이터를 전달한다. 이렇게 하면 일부 백그라운드 연산들과 함께 모델의 forward를 실행한다.

model에 데이터를 전달하면 2차원 텐서를 반환한다. dim=0은 각 분류(class)에 대한 raw predicted values가, dim=1에는 각 출력의 개별적인 값들이 있다.

아래와 같이 이 값을 nn.Softmax에 넘겨주어 확률값을 얻어낼 수 있다.

In [8]:
X = torch.rand(3, 28, 28, device=device)
logits = model(X)
logits, logits.shape

(tensor([[ 0.0389, -0.0472, -0.0112,  0.0729, -0.1464, -0.0405, -0.0185, -0.0261,
          -0.0052, -0.0878, -0.0349,  0.0007,  0.0522, -0.0990,  0.0286,  0.0696,
          -0.0225, -0.0342],
         [ 0.0339, -0.0398, -0.0264,  0.0665, -0.1718,  0.0506, -0.0013, -0.0844,
           0.0395, -0.0820,  0.0079, -0.0066,  0.0618, -0.1236, -0.0275,  0.1001,
          -0.0152, -0.0486],
         [ 0.0561, -0.0814, -0.0153,  0.0646, -0.1164,  0.0562,  0.0037, -0.0288,
          -0.0017, -0.1036,  0.0489,  0.0403,  0.0168, -0.0820, -0.0051,  0.0227,
          -0.0064, -0.0506]], grad_fn=<AddmmBackward0>),
 torch.Size([3, 18]))

In [9]:
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f'Predicted class: {y_pred}')

Predicted class: tensor([ 3, 15,  3])


## Model Layers

FashionMNIST 모델의 layer들을 자세히 살펴보자.

In [16]:
input_image = torch.rand(3, 28, 28)
input_image.size()

torch.Size([3, 28, 28])

nn.Flatten은 28x28 image를 784개의 픽셀 값들을 갖고 있는 contiguous array로 만든다.

In [17]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
flat_image.size()

torch.Size([3, 784])

nn.Linear은 저장된 weights와 bias를 사용해 linear transformation을 적용한다.

In [18]:
layer1 = nn.Linear(in_features=28 * 28, out_features=20)
hidden1 = layer1(flat_image)
hidden1.size()

torch.Size([3, 20])

이 모델에서는 nn.ReLU를 Non-linear 활성화를 위해 사용하는데, 다른 활성화를 도입할 수도 있다.

In [19]:
print(f'Before ReLU: {hidden1}\n\n')
hidden1 = nn.ReLU()(hidden1)
print(f'After ReLU: {hidden1}')

Before ReLU: tensor([[-0.3632,  0.2206, -0.0947, -0.0158, -0.0545, -0.0433,  0.1038, -0.2928,
          0.1661,  0.0125, -0.2600,  0.2028, -0.4580,  0.5020,  0.2187,  0.1670,
         -0.0795,  0.1862,  0.1253,  0.1903],
        [-0.4817, -0.0630, -0.1336,  0.1391, -0.3439, -0.2601, -0.0034, -0.1140,
          0.1843, -0.1564, -0.0600,  0.0504, -0.3888,  0.4677,  0.2818, -0.1439,
          0.0840,  0.3408,  0.4783, -0.0702],
        [-0.4678,  0.0444,  0.2156,  0.0777, -0.3044, -0.1166,  0.0472, -0.2092,
          0.0175,  0.2032, -0.2841, -0.0456, -0.2136,  0.3230,  0.2733,  0.1156,
          0.0054,  0.1817,  0.2453,  0.1578]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.2206, 0.0000, 0.0000, 0.0000, 0.0000, 0.1038, 0.0000, 0.1661,
         0.0125, 0.0000, 0.2028, 0.0000, 0.5020, 0.2187, 0.1670, 0.0000, 0.1862,
         0.1253, 0.1903],
        [0.0000, 0.0000, 0.0000, 0.1391, 0.0000, 0.0000, 0.0000, 0.0000, 0.1843,
         0.0000, 0.0000, 0.0504, 0.0000, 0.4677, 0.28

nn.Sequential은 순서를 갖는 모듈의 컨테이너이다. 데이터는 미리 정해진 순서대로 모듈들에 전달된다.

In [20]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3, 28, 28)
logits = seq_modules(input_image)
logits

tensor([[ 0.0366,  0.1074,  0.2082, -0.0566, -0.1334, -0.1278, -0.0894,  0.1909,
          0.3720,  0.0464],
        [ 0.0390,  0.1764,  0.1176, -0.0997, -0.1956, -0.0984, -0.0989,  0.1648,
          0.2692,  0.0159],
        [ 0.1136,  0.0981,  0.2394, -0.0691, -0.1553, -0.1042, -0.1500,  0.1729,
          0.2494,  0.0121]], grad_fn=<AddmmBackward0>)

nn.Softmax를 통해 [-infty, infty] 범위의 logits를 [0, 1] 범위로 scaling해준다.

In [21]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)
pred_probab

tensor([[0.0969, 0.1040, 0.1150, 0.0883, 0.0817, 0.0822, 0.0854, 0.1131, 0.1355,
         0.0978],
        [0.1000, 0.1147, 0.1081, 0.0870, 0.0791, 0.0871, 0.0871, 0.1134, 0.1258,
         0.0977],
        [0.1064, 0.1048, 0.1207, 0.0886, 0.0813, 0.0856, 0.0817, 0.1129, 0.1219,
         0.0961]], grad_fn=<SoftmaxBackward0>)

## Model Parameters

Neural network 안의 layer들은 parameterize 된다.(즉, weights와 biases가 training 중 최적화된다.)

nn.Module을 상속하면 자동적으로 모델 객체 내의 모든 필드를 추적하고, 이는 parameters()나 named_parameters()를 통해 모든 파라미터에 접근 가능하게 한다.

In [23]:
print(f'Model Structure: {model}\n\n')

for name, param in model.named_parameters():
    print(f'Layer: {name} | Size: {param.size()} | Values: {param[:2]}\n')

Model Structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=18, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values: tensor([[ 0.0225, -0.0315, -0.0043,  ...,  0.0202, -0.0245, -0.0123],
        [-0.0168, -0.0316, -0.0354,  ..., -0.0278, -0.0066,  0.0245]],
       grad_fn=<SliceBackward0>)

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values: tensor([-0.0043,  0.0037], grad_fn=<SliceBackward0>)

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values: tensor([[ 0.0147,  0.0035, -0.0432,  ...,  0.0256, -0.0109,  0.0319],
        [ 0.0285, -0.0317,  0.0194,  ..., -0.0269,  0.0132, -0.0304]],
       grad_fn=<SliceBackward0>)

Layer: linear_relu_stack.2.bias | Size: 